In [ ]:
# extract all posts & rename them with variation #
from pathlib import Path
import shutil


def restructure_variation_folders(input_folder: str) -> tuple[int, list[str]]:
    root = Path(input_folder)
    if not root.exists():
        raise ValueError(f"Folder not found: {input_folder}")

    successful_moves = 0
    errors = []

    # More flexible variation folder matching
    variation_folders = [
        f
        for f in root.iterdir()
        if f.is_dir() and f.name.lower().startswith("variation")
    ]

    print(
        f"Found variation folders: {[f.name for f in variation_folders]}"
    )  # Debug line

    for var_folder in variation_folders:
        try:
            # Extract variation number more flexibly
            var_num = int("".join(filter(str.isdigit, var_folder.name)))

            # Find all post folders
            post_folders = [
                f
                for f in var_folder.iterdir()
                if f.is_dir() and f.name.startswith("post")
            ]

            print(
                f"Found post folders in {var_folder.name}: {[f.name for f in post_folders]}"
            )  # Debug line

            for post_folder in post_folders:
                try:
                    # Extract post number more flexibly
                    post_num = int("".join(filter(str.isdigit, post_folder.name)))

                    # Create new folder name
                    new_name = root / f"post_{post_num}_{var_num}"

                    print(f"Moving {post_folder} to {new_name}")  # Debug line

                    # Move folder
                    shutil.move(str(post_folder), str(new_name))
                    successful_moves += 1

                except Exception as e:
                    errors.append(f"Error moving {post_folder}: {str(e)}")

            # Remove empty variation folder
            var_folder.rmdir()

        except Exception as e:
            errors.append(f"Error processing {var_folder}: {str(e)}")

    return successful_moves, errors


# Run the restructuring
moved, errors = restructure_variation_folders("/Users/juri/Downloads/moreoutput")

# Check results
print(f"Successfully moved {moved} folders")
if errors:
    print("Errors encountered:")
    for error in errors:
        print(f"  - {error}")

In [ ]:
# group subfolders into N groups
from pathlib import Path
import shutil


def group_posts(input_folder: str, group_size: int) -> tuple[int, list[str]]:
    """
    Groups post folders into groups of specified size.

    Args:
        input_folder: Path to folder containing post_X_Y folders
        group_size: Number of posts per group

    Returns:
        tuple: (number of successful moves, list of errors)
    """
    root = Path(input_folder)
    if not root.exists():
        raise ValueError(f"Folder not found: {input_folder}")

    successful_moves = 0
    errors = []

    # Get all post folders
    post_folders = sorted(
        [f for f in root.iterdir() if f.is_dir() and f.name.startswith("post_")]
    )

    print(f"Found {len(post_folders)} post folders")

    # Calculate number of groups needed
    num_groups = (len(post_folders) + group_size - 1) // group_size

    # Create groups and move folders
    for group_num in range(1, num_groups + 1):
        try:
            # Create group folder
            group_folder = root / f"group_{group_num}"
            group_folder.mkdir(exist_ok=True)

            # Get slice of posts for this group
            start_idx = (group_num - 1) * group_size
            end_idx = min(start_idx + group_size, len(post_folders))
            group_posts = post_folders[start_idx:end_idx]

            print(f"Moving {len(group_posts)} posts to group_{group_num}")

            # Move each post to group folder
            for post in group_posts:
                try:
                    shutil.move(str(post), str(group_folder / post.name))
                    successful_moves += 1
                except Exception as e:
                    errors.append(f"Error moving {post.name}: {str(e)}")

        except Exception as e:
            errors.append(f"Error creating group {group_num}: {str(e)}")

    return successful_moves, errors


# Group posts into groups of 4
moved, errors = group_posts("/Users/juri/Downloads/moreoutput", 4)

# Check results
print(f"Successfully moved {moved} folders")
if errors:
    print("Errors encountered:")
    for error in errors:
        print(f"  - {error}")

In [ ]:
# zip sub folders in a folder
from pathlib import Path
import shutil
import zipfile
import logging


def zip_groups(input_folder: str, individual: bool = False) -> tuple[int, list[str]]:
    """
    Zip group folders either individually or into one archive.

    Args:
        input_folder: Path to folder containing group folders
        individual: If True, creates separate zip for each group.
                   If False, creates one zip containing all groups.

    Returns:
        tuple: (number of successful zips, list of errors)
    """
    root = Path(input_folder)
    if not root.exists():
        raise ValueError(f"Folder not found: {input_folder}")

    successful = 0
    errors = []

    # Find all group folders
    group_folders = sorted(
        [f for f in root.iterdir() if f.is_dir() and f.name.startswith("group_")]
    )

    if not group_folders:
        raise ValueError("No group folders found")

    if individual:
        # Zip each group separately
        for group in group_folders:
            try:
                zip_path = root / f"{group.name}.zip"
                with zipfile.ZipFile(zip_path, "w", zipfile.ZIP_DEFLATED) as zipf:
                    # Walk through all files in group folder
                    for item in group.rglob("*"):
                        if item.is_file():
                            # Calculate relative path for zip structure
                            relative_path = item.relative_to(group)
                            zipf.write(item, arcname=f"{group.name}/{relative_path}")
                successful += 1
            except Exception as e:
                errors.append(f"Error zipping {group.name}: {str(e)}")

    else:
        # Zip all groups into one file
        try:
            zip_path = root / "all_groups.zip"
            with zipfile.ZipFile(zip_path, "w", zipfile.ZIP_DEFLATED) as zipf:
                for group in group_folders:
                    # Walk through all files in group folder
                    for item in group.rglob("*"):
                        if item.is_file():
                            # Calculate relative path for zip structure
                            relative_path = item.relative_to(root)
                            zipf.write(item, arcname=relative_path)
            successful += 1
        except Exception as e:
            errors.append(f"Error creating combined zip: {str(e)}")

    return successful, errors


# Zip each group into separate files
successful, errors = zip_groups("/Users/juri/Downloads/moreoutput", individual=True)

# Or zip all groups into one file
# successful, errors = zip_groups("/path/to/folder", individual=False)

# Check results
print(f"Successfully created {successful} zip files")
if errors:
    print("Errors encountered:")
    for error in errors:
        print(f"  - {error}")

In [ ]:
# convert folder images to png

from pathlib import Path
from PIL import Image
import logging


def convert_to_png(
    folder_path: str, output_folder: str = None
) -> tuple[int, list[str]]:
    """
    Convert all images in a folder to PNG format.

    Args:
        folder_path: Path to folder containing images
        output_folder: Optional path to output folder. If None, uses same folder

    Returns:
        tuple: (number of successful conversions, list of failed files)
    """
    folder = Path(folder_path)
    if not folder.exists():
        raise ValueError(f"Folder not found: {folder_path}")

    if output_folder:
        out_folder = Path(output_folder)
        out_folder.mkdir(parents=True, exist_ok=True)
    else:
        out_folder = folder

    converted = 0
    failed = []

    # Common image extensions to process
    valid_extensions = {".jpg", ".jpeg", ".bmp", ".tiff", ".gif"}

    for file_path in folder.iterdir():
        if file_path.suffix.lower() in valid_extensions:
            try:
                # Open and convert image
                with Image.open(file_path) as img:
                    # Create new filename with .png extension
                    new_name = out_folder / f"{file_path.stem}.png"
                    # Convert and save
                    img.save(new_name, "PNG")
                converted += 1
            except Exception as e:
                failed.append(f"{file_path.name}: {str(e)}")

    return converted, failed


# Basic usage (same folder)
converted, failed = convert_to_png("/Users/juri/Downloads/productreviews/shill")

# Check results
print(f"Successfully converted {converted} images")
if failed:
    print("Failed conversions:")
    for f in failed:
        print(f"  - {f}")